# 自动求导
之前的文档写的太过于简单了，现在又忘记了，所以再根据教程敲了一个。

[Pytorch自动求导机制的详解](https://blog.csdn.net/weixin_44023658/article/details/107417063)

In [1]:
import torch as t 

In [2]:
# tensor 的 require_grad属性
x= t.tensor(3.0,requires_grad=True)
y = t.tensor(4.,requires_grad=False)
z= t.pow(x,2)+t.pow(y,2)

print(x.requires_grad)
print(y.requires_grad)
print(z.requires_grad)

True
False
True


In [3]:
z.backward()
x.grad,y.grad

(tensor(6.), None)

In [ ]:
# leaf variable 可导则z可导，这里我们x是可导的，所以就有梯度，如果x变成不可导的，那么就不行了

#  默认的求导规则
只能标量对标量进行求导、或者是标量对矩阵进行求导。

前者就是我们上面的例子了，后者一般是我们对于损失函数进行求导，损失函数一般是进行sum或者mean之后的一个标量，然而我们神经网络中的参数一般都是矩阵，所以这时候需要对矩阵进行求导。

在下面我们要验证`标量对于矩阵求导`的正确性

$$\mathbf{Y} = \mathbf{W}  \mathbf{x} + \mathbf{b}$$

In [6]:
X =t.tensor([1.5,2.,3.],requires_grad=False)
# 由于w b是线性层的参数，我们需要追踪它们的梯度，所以设置require_grad
W = t.tensor([0.1,0.2,0.3],requires_grad=True)
b= t.tensor(0.111,requires_grad=True)
Y = X*W+b
Y=Y.sum()

Y

tensor(1.7830, grad_fn=<SumBackward0>)

In [7]:
# 进行bp,W的梯度应该是1.5,2.,3.，b的梯度应该是1
Y.backward()
W.grad,b.grad


(tensor([1.5000, 2.0000, 3.0000]), tensor(3.))

# 多维度矩阵和复合函数
上面已经成功验证了，没毛病，现在对于多维度的矩阵和复合函数进行试验
$$
\begin{aligned}\begin{split}
\mathbf{Y} & = 1+\mathbf{X}\\
\mathbf{Z} & = \mathbf{Y}^2\\
\end{split}\end{aligned}
$$

In [9]:
X = t.tensor([[1.,2.,3.,],[4.,5.,6.]],requires_grad=True)
Y = X +1 
Z = 2*Y**2
F = Z.mean()


In [10]:
# 查看是否需要梯度计算
print(
    X.requires_grad,
    Y.requires_grad,
    Z.requires_grad,
    F.requires_grad
)

True True True True


In [11]:
F.backward()
X.grad

tensor([[1.3333, 2.0000, 2.6667],
        [3.3333, 4.0000, 4.6667]])

# 矩阵对矩阵求导
需要通过传入gradient来实现

In [20]:
X = t.tensor([[1.,2.,3.,],[4.,5.,6.]],requires_grad=True)
Y  =X + X**2
gradient = t.tensor([[1.,1.,1.],[1.,1.,1.]])
# 在这里我们是对于Y的各个元素相对应的X进行求导
Y.backward(gradient,retain_graph=True)

X.grad
#清除梯度
X.grad.zero_()


tensor([[0., 0., 0.],
        [0., 0., 0.]])

In [21]:
# 如果我们修改一下gradient会发生什么呢？
gradient = t.tensor([[1.,100.,1.],[1.,1.,1.]])
Y.backward(gradient)
X.grad



tensor([[  3., 500.,   7.],
        [  9.,  11.,  13.]])

可以看到，最后求出来的grad是根据gradient相乘而得到的。
1. gradient.shape = Y.shape
2. 每一个元素表示Y对应位置的元素所对应的权重


# Retrain Graph And Create Graph
如果我们有以下的计算:

$$
\begin{split}\begin{aligned}
y = f_1(x)\\
z= f_2(x)\\
p=g_1(y)\\
q=g_2(z)\\
\end{aligned}\end{split}
$$

一个计算图在进行反向传播求导之后，它就销毁了，如果还要求导就会报错了，除非你知道你在做什么，否则不要去用它。

在上面的例子中，我们对p进行求导，然后计算图就被销毁了，然后不论是调用q.backward()还是p.backward()都会报错。

用处是在强化学习中更新Actor的时候会用到。
